In [ ]:
import numpy as np
import sympy as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('./data/dairy_nutrition.csv')
df

In [ ]:
df.drop(columns='Description', inplace=True)
df

## Part (a)

PCA should be carried out on the correlation matrix. All the variables (nutrients) are measured in terms of mass. Although they have different units, they can be scaled such that they are all in grams.

## Part (b)

In [ ]:
df['VitA_g'] = df['VitA_mcg'] * 1e-6
df.drop(columns='VitA_mcg', inplace=True)
df

In [ ]:
df['Calcium_g'] = df['Calcium_mg'] * 1e-3
df.drop(columns='Calcium_mg', inplace=True)
df

In [ ]:
df_numeric = df.iloc[:,1:]

In [ ]:
df_corr = df_numeric.corr()
df_corr

In [ ]:
pca = PCA(n_components=6).fit(df_numeric)

In [ ]:
def pca_results(data, pca):
    
    # Dimension indexing
    dimensions = [f'PC {i}' for i in range(1, len(pca.components_) + 1)]
    
    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = data.keys()) 
    components.index = dimensions

    #PCA eigenvalues
    ev = pca.explained_variance_.reshape(len(pca.components_), 1)
    eigenvalues = pd.DataFrame(np.round(ev, 4), columns = ['Eigenvalue']) 
    eigenvalues.index = dimensions
    
    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1) 
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance']) 
    variance_ratios.index = dimensions

    cum_ratios = np.cumsum(ratios)
    cum_variance_ratios = pd.DataFrame(np.round(cum_ratios, 4), columns = ['Cumulative Explained Variance']) 
    cum_variance_ratios.index = dimensions

    # Return a concatenated DataFrame
    return pd.concat([eigenvalues, variance_ratios, cum_variance_ratios, components], axis = 1)

pca_results(df_numeric, pca)

In [ ]:
def corr_pca(df_corr):
    eig_vals, eig_vecs = np.linalg.eig(df_corr)

    pca_results_ = pd.DataFrame(
        data=np.hstack((eig_vals.reshape(-1, 1), eig_vecs)),
        columns=[
            'Eigenvalue',
            'Protein_g', 'Fat_g', 'Carb_g', 'Sugar_g', 'VitA_g', 'Calcium_g']
    ).sort_values(
        by='Eigenvalue',
        ascending=False
    )

    pca_results_['Explained Variance'] = pca_results_['Eigenvalue'] / df_corr.shape[0]

    pca_results_['Cumulative Explained Variance'] = np.cumsum(pca_results_['Explained Variance'])

    pca_results_.index = [f'PC {i + 1}' for i in range(pca_results_.shape[0])]

    return pca_results_[['Eigenvalue', 'Explained Variance', 'Cumulative Explained Variance', 'Protein_g', 'Fat_g', 'Carb_g', 'Sugar_g', 'VitA_g', 'Calcium_g']].round(4)

pca_res = corr_pca(df_corr)

In [ ]:
pca_res

In [ ]:
def eig_expl(pca, proportion=0.8):
    cum_expl_var = pca['Cumulative Explained Variance']
    first_to_cross_threshold = cum_expl_var[cum_expl_var < proportion].shape[0]
    pca_satisified = pca.iloc[:first_to_cross_threshold + 1]
    return pca_satisified

eig_expl(pca_res)

In [ ]:
def eig_more_1(pca):
    return pca[pca['Eigenvalue'] >= 1]

eig_more_1(pca_res)

In [ ]:
def scree_plot(pca):
    with sns.axes_style(style='darkgrid'):
        ax = sns.pointplot(x=pca.index, y=pca['Eigenvalue'])
        ax.set(
            title='Scree Plot'
        )
        ax.plot(2, pca['Eigenvalue'][2],
            marker='o',
            mec='r',
            mfc='none',
            markersize=28
        )

scree_plot(pca_res)

In [ ]:
pca_res = pca_res.iloc[:2,:]
pca_res

In [ ]:
# def original(df, pca):
    
#     pc1 = pca.components_
#     PC1 = df @ pc1

#     pc2 = pca.components_.T
#     PC2 = df @ pc2

#     print(PC1)

#     ax = sns.scatterplot(x=PC1, y=PC2)
#     ax.set(
#         title='Score Plot',
#     )
#     ylim = ax.get_ylim()
#     xlim = ax.get_xlim()
#     ax.plot([0, 0], [ylim[0], ylim[1]], color='grey', linestyle='--', linewidth=1)
#     ax.plot([xlim[0], xlim[1]], [0, 0], color='grey', linestyle='--', linewidth=1)

# original(df_numeric, pca)

In [ ]:
def score_plot(df, pca, centre=False):
    
    pc1 = pca.loc['PC 1'].iloc[3:]
    PC1 = df @ pc1 if not centre else (df - df.mean(axis=0)) @ pc1

    pc2 = pca.loc['PC 2'].iloc[3:]
    PC2 = df @ pc2 if not centre else (df - df.mean(axis=0)) @ pc2

    ax = sns.scatterplot(x=PC1, y=PC2)
    ax.set(
        title='Score Plot',
    )
    ylim = ax.get_ylim()
    xlim = ax.get_xlim()
    ax.plot([0, 0], [ylim[0], ylim[1]], color='grey', linestyle='--', linewidth=1)
    ax.plot([xlim[0], xlim[1]], [0, 0], color='grey', linestyle='--', linewidth=1)

score_plot(df_numeric, pca_res, centre=True)

In [ ]:
def loading_plot_R(data, pca, width=5, height=5, margin=0.5):

    fig, ax = plt.subplots(figsize = (width,height))

    #Set limits for figure
    x_min = min(pca.loc['PC 1'].min(),0)-margin
    x_max = max(pca.loc['PC 1'].max(),0)+margin
    y_min = min(pca.loc['PC 2'].min(),0)-margin
    y_max = max(pca.loc['PC 2'].max(),0)+margin

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    #Scaling factor for text position
    text_x_adj = -0.1
    text_y_adj = 0.2

    for i, e in enumerate(pca):
        ax.arrow(0, 0, pca_res[e]['PC 1'], pca_res[e]['PC 2'], head_width=0.1, head_length=0.1, linewidth=2, color='red')
        ax.text(pca_res[e]['PC 1'] + text_x_adj, pca_res[e]['PC 2'] + text_y_adj, data.columns[i], color='black', ha='center', va='center', fontsize=12)

    plt.plot([x_min, x_max], [0, 0], color='k', linestyle='--', linewidth=1)
    plt.plot([0, 0], [y_min, y_max], color='k', linestyle='--', linewidth=1)
    ax.set_xlabel("PC1", fontsize=14)
    ax.set_ylabel("PC2", fontsize=14)
    ax.set_title("Loading plot", fontsize = 14)

loading_plot_R(df_numeric, pca_res.iloc[:,3:])

In [ ]:
# This function plots the loading plot.
# Pass original data dataframe and returns of PCA to this function. Optional width, height and margin
# This function returns the axes of the loading plot

def loading_plot_C(data, pca, width=5, height=5, margin=0.5):

    fig, ax = plt.subplots(figsize = (width,height))

    #Set limits for figure
    x_min = min(pca.components_[0,:].min(),0)-margin
    x_max = max(pca.components_[0,:].max(),0)+margin
    y_min = min(pca.components_[1,:].min(),0)-margin
    y_max = max(pca.components_[1,:].max(),0)+margin

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    #Scaling factor for text position
    text_pos = 0.2

    for i, v in enumerate(pca.components_.T):
        ax.arrow(0, 0, v[0], v[1], head_width=0.1, head_length=0.1, linewidth=2, color='red')
        ax.text(v[0], v[1]+text_pos, data.columns[i], color='black', ha='center', va='center', fontsize=12)

    plt.plot([x_min, x_max], [0, 0], color='k', linestyle='--', linewidth=1)
    plt.plot([0, 0], [y_min, y_max], color='k', linestyle='--', linewidth=1)
    ax.set_xlabel("PC1", fontsize=14)
    ax.set_ylabel("PC2", fontsize=14)
    ax.set_title("Loading plot", fontsize = 14)

loading_plot_C(df_numeric, pca)

In [ ]:
def score_plot_labels(df, df_numeric, pca):
    pc1 = pca.loc['PC 1'].iloc[3:]
    PC1 = (df_numeric - df_numeric.mean(axis=0)) @ pc1

    pc2 = pca.loc['PC 2'].iloc[3:]
    PC2 = (df_numeric - df_numeric.mean(axis=0)) @ pc2

    comb = pd.concat(objs=(df['Type'], PC1, PC2), axis=1)
    comb.columns = ['Type', 'PC1', 'PC2']

    ax = sns.scatterplot(data=comb, x='PC1', y='PC2', hue='Type')
    ax.set(
        title='Score Plot'
    )
    ylim = ax.get_ylim()
    xlim = ax.get_xlim()
    ax.plot([0, 0], [ylim[0], ylim[1]], color='grey', linestyle='--', linewidth=1)
    ax.plot([xlim[0], xlim[1]], [0, 0], color='grey', linestyle='--', linewidth=1)

score_plot_labels(df, df_numeric, pca_res)

## Part (c)